In [2]:
## IMPORTS
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
from datetime import date, timedelta
from yahoo_fin.stock_info import get_data

In [ ]:
## Constant
SHORT_PEAK = 16
SHORT_TROUGH = 17
RESISTANCE = 15


In [ ]:
goodSector = pd.DataFrame()
spdf = pd.read_pickle("stockData/Spy.pkl")
sectorOfTicker = pd.read_pickle("stockData/nasdaq.pkl")
sectorOfNyse = pd.read_pickle("stockData/nyse.pkl")
sectorOfTicker.update(sectorOfNyse)

## TOOL FUNCTIONS
def fullPrint(df):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
        print(df)



In [2]:
def checkIfStage4(
    i,price,volumePerc, RS, slope, wMA30,prevStage,prevClose,prevResistance,peak,prevPeak,prevTrough,index,dfSorted,secondBought,initialSupport,fiveYearHigh,param,goodSectorDf):
    if prevStage == "Stage 4" or prevStage == "Sale Short":
        if price < prevPeak:
            # update shortPeak and shortTrough
            dfSorted.iat[i,SHORT_PEAK] = price
            dfSorted.iat[i,SHORT_TROUGH] = price
        else:
            dfSorted.iat[i,SHORT_PEAK] = prevPeak
            dfSorted.iat[i,SHORT_TROUGH] = min(prevTrough, price)
        if price >= prevResistance*param[5] or price > wMA30*param[6]:
            return "Cover Buy"

        dfSorted.iat[i,RESISTANCE] = prevResistance

        if price == dfSorted.iat[i,SHORT_TROUGH] and prevResistance < prevTrough*param[6]:
            dfSorted.iat[i,RESISTANCE] = prevPeak
        return "Stage 4"
        
    if price > wMA30*param[0]:
        return "Price"
    if slope > param[1]:
        return "Slope"
    if volumePerc < param[2]:
        return "volume"
    if RS > param[3]:
        return "RS"
    if price < prevClose:
        dfSorted.iat[i,SHORT_PEAK] = price
        dfSorted.iat[i,SHORT_TROUGH] = price
    else:
        dfSorted.iat[i,SHORT_PEAK] = price
        dfSorted.iat[i,SHORT_TROUGH] = price*0.95
    
    return "Sale Short"
  